In [2]:
import tensorflow as tf
import pandas as pd
import glob
import json
import matplotlib.pyplot as plt
import numpy as np
import random
import os
import tensorflow_io as tfio

In [3]:
def seed_it_all(seed=7):
    """ Attempt to be Reproducible """
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

seed_it_all()

In [4]:
class CFG:
    left_ROWS_per_frame = 21
    sequence_length = 30
    batch_size = 32

labels  = json.load(open('sign_to_prediction_index_map.json','r'))
complete_df = pd.read_csv('train.csv')
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(complete_df, test_size=0.2)

In [4]:
def build_loader(with_labels=True):
    def load_video(video_path):
        #print('herer')
        video_df = tfio.IODataset.from_parquet(video_path)
        #video_df = pd.read_parquet(video_path, engine='pyarrow')
        #video_df.fillna(0,inplace=True)
        left_df = video_df[video_df.type=='left_hand']
        left_values = left_df[['x','y','z']].values
        left_values = left_values.reshape(-1,CFG.left_ROWS_per_frame,3)
        left_hand_array =  tf.image.resize(left_values, (CFG.sequence_length, CFG.left_ROWS_per_frame))
        right_df = video_df[video_df.type=='right_hand']
        right_values = right_df[['x','y','z']].values
        right_values = right_values.reshape(-1,CFG.left_ROWS_per_frame,3)
        right_hand_array =  tf.image.resize(right_values, (CFG.sequence_length, CFG.left_ROWS_per_frame))
        return [left_hand_array, right_hand_array]
    
    def load_video_with_labels(path, label):
        return load_video(path), labels[label]
    
    return load_video_with_labels if with_labels else load_video

In [5]:
class CustomData(tf.keras.utils.Sequence):
    def __init__(self,df,num_frames=20,batch_size=8,shuffle=True,\
                 labels_path='sign_to_prediction_index_map.json'):
        self.df = df
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.num_frames = num_frames
        self.labels  = json.load(open('sign_to_prediction_index_map.json','r'))
        self.on_epoch_end()
        
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.df))
        if self.shuffle:
            np.random.shuffle(self.indexes)
    
    def __getitem__(self,index):
        batches = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        left_hand_input = np.zeros(shape=(self.batch_size,self.num_frames,CFG.left_ROWS_per_frame,3))
        right_hand_input = np.zeros(shape=(self.batch_size,self.num_frames,CFG.left_ROWS_per_frame,3))
        labels = []
        for i,row_val in enumerate(batches):
            row = self.df.iloc[row_val]
            left_hand,right_hand = self.load_video(row['path'])
            left_hand_input[i,:] = left_hand
            right_hand_input[i,:] = right_hand
            labels.append(self.labels[row['sign']])
        return [left_hand_input,right_hand_input],np.asarray(labels)
            
    def load_video(self,video_path):
        video_df = pd.read_parquet(video_path, engine='pyarrow')
        video_df.fillna(0,inplace=True)
        left_df = video_df[video_df.type=='left_hand']
        left_values = left_df[['x','y','z']].values
        left_values = left_values.reshape(-1,CFG.left_ROWS_per_frame,3)
        left_hand_array =  tf.image.resize(left_values, (CFG.sequence_length, CFG.left_ROWS_per_frame))
        right_df = video_df[video_df.type=='right_hand']
        right_values = right_df[['x','y','z']].values
        right_values = right_values.reshape(-1,CFG.left_ROWS_per_frame,3)
        right_hand_array =  tf.image.resize(right_values, (CFG.sequence_length, CFG.left_ROWS_per_frame))
        right_hand_array[:,]
        return left_hand_array, right_hand_array
    
    def __len__(self):
        return len(self.df)//self.batch_size

In [6]:
train_datagen = CustomData(train_df,num_frames=CFG.sequence_length,batch_size=CFG.batch_size)
test_datagen = CustomData(test_df,num_frames=CFG.sequence_length,batch_size=CFG.batch_size)

In [6]:
video_df = pd.read_parquet('train_landmark_files/16069/100015657.parquet', engine='pyarrow')
video_df.fillna(0,inplace=True)
left_df = video_df[video_df.type=='left_hand']
left_values = left_df[['x','y','z']].values
left_values = left_values.reshape(-1,CFG.left_ROWS_per_frame,3)
left_hand_array =  tf.image.resize(left_values, (CFG.sequence_length, CFG.left_ROWS_per_frame))
right_df = video_df[video_df.type=='right_hand']
right_values = right_df[['x','y','z']].values
right_values = right_values.reshape(-1,CFG.left_ROWS_per_frame,3)
right_hand_array =  tf.image.resize(right_values, (CFG.sequence_length, CFG.left_ROWS_per_frame))

In [18]:
# Visualization Imports (overkill)
from matplotlib.animation import FuncAnimation
from matplotlib.colors import ListedColormap
from matplotlib.patches import Rectangle
import matplotlib.patches as patches
import plotly.graph_objects as go
from IPython.display import HTML
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm; tqdm.pandas();
import plotly.express as px
import tifffile as tif
import seaborn as sns
from PIL import Image, ImageEnhance; Image.MAX_IMAGE_PIXELS = 5_000_000_000;
import matplotlib; print(f"\t\t– MATPLOTLIB VERSION: {matplotlib.__version__}");
from matplotlib import animation, rc; rc('animation', html='jshtml')
import plotly
import PIL
import cv2

		– MATPLOTLIB VERSION: 3.6.2


In [21]:
## Change this directory to any file
path_to_sign = 'train_landmark_files/16069/1011655866.parquet'
sign = pd.read_parquet(path_to_sign)
sign.y = sign.y * -1

In [22]:
video_df = pd.read_parquet('train_landmark_files/32319/1001258102.parquet', engine='pyarrow')
video_df.dropna(inplace=True)
left_df = video_df[video_df.type=='left_hand']
left_df.y = left_df.y * -1
left_values = left_df[['x','y','z']].values
left_values = left_values.reshape(-1,CFG.left_ROWS_per_frame,3)
left_resized =  tf.image.resize(left_values, (30, CFG.left_ROWS_per_frame)).numpy()

<ipython-input-22-34b809472400>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  left_df.y = left_df.y * -1


In [20]:
new_norm[:,:,:1] = 1 - new_norm[:,:,:1] 

In [23]:
new_norm = left_resized.copy()
new_norm[:,:,0] = (new_norm[:,:,0]- new_norm[:,:,0].min())/(new_norm[:,:,0].max()- new_norm[:,:,0].min())
new_norm[:,:,1] = (new_norm[:,:,1]- new_norm[:,:,1].min())/(new_norm[:,:,1].max()- new_norm[:,:,1].min())
new_norm[:,:,2] = (new_norm[:,:,2]- new_norm[:,:,2].min())/(new_norm[:,:,2].max()- new_norm[:,:,2].min())

In [6]:
X = np.load('gislr-dataset-public/X.npy')
y = np.load('gislr-dataset-public/y.npy')
NON_EMPTY_FRAME_IDXS = np.load('gislr-dataset-public/NON_EMPTY_FRAME_IDXS.npy')

In [37]:
left_x = X[2:3,:,40:61,:][0]
new_left_x = left_x.copy()
new_left_x[:,:,:1] = np.max(left_x[:,:,:1]) - left_x[:,:,:1]

In [19]:
def get_hand_points(hand):
    x = [[hand[0,0], hand[1,0], hand[2,0], hand[3,0], hand[4,0]], # Thumb
         [hand[5,0], hand[6,0], hand[7,0], hand[8,0]], # Index
         [hand[9,0], hand[10,0], hand[11,0], hand[12,0]], 
         [hand[13,0], hand[14,0], hand[15,0], hand[16,0]], 
         [hand[17,0], hand[18,0], hand[19,0], hand[20,0]], 
         [hand[0,0], hand[5,0], hand[9,0], hand[13,0], hand[17,0], hand[0,0]]]

    y = [[hand[0,1], hand[1,1], hand[2,1], hand[3,1], hand[4,1]],  #Thumb
         [hand[5,1], hand[6,1], hand[7,1], hand[8,1]], # Index
         [hand[9,1], hand[10,1], hand[11,1], hand[12,1]], 
         [hand[13,1], hand[14,1], hand[15,1], hand[16,1]], 
         [hand[17,1], hand[18,1], hand[19,1], hand[20,1]], 
         [hand[0,1], hand[5,1], hand[9,1], hand[13,1], hand[17,1], hand[0,1]]] 
    return x, y

def complete_video(left_values):
    def animation_frame(f):
        left = left_values[f]
        lx, ly = get_hand_points(left)
        ax.clear()
        for i in range(len(lx)):
            ax.plot(lx[i], ly[i])
        plt.xlim(xmin, xmax)
        plt.ylim(ymin, ymax)


    #print(f"The sign being shown here is: {train[train.path==f'{path_to_sign}'].sign.values[0]}")

    ## These values set the limits on the graph to stabilize the video
    xmin = left_values[:,:,0].min() - 0.2
    xmax = left_values[:,:,0].max() + 0.2
    ymin = left_values[:,:,1].min() - 0.2
    ymax = left_values[:,:,1].max() + 0.2

    fig, ax = plt.subplots()
    l, = ax.plot([], [])
    animation = FuncAnimation(fig, func=animation_frame, frames=np.arange(0,len(left_values)))

    return HTML(animation.to_html5_video())


In [36]:
new_left_x.shape

(32, 21, 1)

In [39]:
complete_video(left_x)

In [21]:
complete_video(new_norm)